# Create an Azure AI Foundry Agent using Semantic Kernel


**Step 1**: Install NuGet packages

To get started with Azure AI Foundry, you need to install the required NuGet packages:

In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [10]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\ai-foundry\../..\.env


**Step 3**: Instantiate the Project client

Create an instance of the `PersistentAgentsClient` using the endpoint URL and the Azure AI Foundry project.

Note that **DefaultAzureCredential** is used for authentication. To successfully authenticate, log in to the Azure CLI using `az login --use-device-code`.

In [11]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;


var projectEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
        var client = AzureAIAgent.CreateAgentsClient(projectEndpoint, new DefaultAzureCredential());
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 4**: Define your agent

In this step, we define the attributes for the agent, including its name, instructions, tools (if any), and the model to be used. 

In [12]:
var modelId = Environment.GetEnvironmentVariable("MODEL");
var agentName = "WeatherAgent";
var agentInstructions = @"
You are a weather assistant. Use the WeatherPlugin to answer questions about the weather in any city.
If the user asks for other information, politely decline and suggest they ask about the weather instead.
If the city is not specified, ask the user to provide a city name.
If the weather data is not available, inform the user that the weather information could not be retrieved.";

var agentDefinition = await client.Administration.CreateAgentAsync(
    modelId,
    name: agentName,
    instructions: agentInstructions,
    tools: [],
    toolResources: null);

**Step 5**: Create the agent

Finally, we create the agent using the `CreateAgentAsync` method of the `PersistentAgentsClient`. This method takes the agent definition and returns the created agent.

In [13]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var agent = new AzureAIAgent(agentDefinition, client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 5**: Creat a new thread for the agent

In [14]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var thread = new AzureAIAgentThread(client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 6** : Create a new message for the agent

In [ ]:
    #pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

    var invokeAgentAsync = async (AzureAIAgentThread agentThread, string input) =>
    {
        ChatMessageContent message = new(AuthorRole.User, input);
        await foreach (ChatMessageContent response in agent.InvokeAsync(message, agentThread))
        {
            Console.WriteLine($"{response.Role}: {response.Content}");
        }
    };

    await invokeAgentAsync(thread, "What is the weather in Australia?");
    #pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

assistant: Australia is a large country, so weather conditions can vary greatly depending on the city or region. Could you please specify a city in Australia? For example, Sydney, Melbourne, Perth, or Brisbane?


**Step 7**: Create additional messages to the same thread

In [16]:
 await invokeAgentAsync(thread, "How about Paris?");

assistant: It seems there was an issue retrieving the weather information for Paris. I apologize for the inconvenience. Could you please try asking again?


**Step 8**: Clean up

Delete the agent and the thread to clean up resources.

In [17]:
await thread.DeleteAsync();
await client.Administration.DeleteAgentAsync(agent.Id);